<h1 style="font-size:40px;"> Dataframe: A fondo </h1>

![](img/lrg.jpg)

&nbsp;   


Vamos a ver con más detalle qué podemos hacer con la API `DataFrame` de spark. Dos buenas referencias son la [documentación oficial](https://spark.apache.org/docs/latest/sql-programming-guide.html) y el libro [*Learning Spark*](http://shop.oreilly.com/product/0636920028512.do)


Empezamos iniciando la sesión:

In [1]:
import os
import pandas as pd

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
conf = (

    SparkConf()
    .setAppName(u"[ICAI] DataFrame: A fondo")
    .set("spark.jars","/var/lib/sqoop/mysql-connector-java-5.1.44-bin.jar")

)

In [3]:
spark = (

    SparkSession.builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()

)

# RDD vs Dataframe

En el anterior sesión vimos que trabajar con `DataFrames` es en general más rápido que trabajar con RDD (y  más sobre todo si trabajamos con *Pyspark*, pero ¿Por qué?




![](img/dataframe.png)
<center>
    https://databricks.com/blog/2015/02/17/introducing-dataframes-in-spark-for-large-scale-data-science.html
</center>

Básicamente al trabajar con datos estructurados se puede usar compresores específicos para los tipos normales de una tabla y además se puede saber qué se está haciendo en cada operación y se pude preeveer el resultado final de cada operación, en spark hay dos proyectos que hacen esto posible:

* **Project Tungsten**: https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html
* **Spark SQL’s Catalyst Optimizer**: https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html

![](img/catalyst.png)

### Trabajando con  RDDs y DFs

Podemos convertir de `RDD` a `DadaFrame` y viceversa con gran facilidad:

In [4]:
lines = spark.sparkContext.textFile("/datos/people.txt")

In [5]:
lines.take(4)

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [6]:
parts = lines.map(lambda l: l.split(",")) #Lo dividimos por comas

#### de RDD a DF:

Usaremos `Row` para definir los registros del RDD como estructurados (filas):

In [7]:
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

In [8]:
people.take(4) #AUN ES UN RDD

[Row(age=29, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [10]:
schemaPeople = people.toDF() #ESTO MAS LO DE ROW LO PASA A DARA FRAME

In [8]:
schemaPeople.show()

+---+-------+
|age|   name|
+---+-------+
| 29|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+



#### de DF A RDD:

In [11]:
teenagers =  schemaPeople.filter('age between 13 and 19').select('name')

In [12]:
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



Con `.rdd` obtenemos el RDD que hay dentro del DataFrame:

In [11]:
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()

In [12]:
for name in teenNames:
    print(name)

Name: Justin


## Funciones

![](img/relation-not-function.gif)

Dentro de la API de `DataFrame` hay multitud de funciones que podemos usar para nuestros análisis. Podemos ver todas ellas en la [documetanción](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions).

Veremos algunos ejemplos basados en siguiente artículo de Databricks:
https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

####  1. Random Data Generation

In [13]:
df = spark.range(0, 10)

In [14]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



F.rand da numeros aleatorios entre 0 y 1, F.randn da normales:

In [15]:
df = df.select("id", F.rand(seed=10).alias("uniform"), F.randn(seed=27).alias("normal"))

In [16]:
df.show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.41371264720975787|  0.5888539012978773|
|  1| 0.7311719281896606|  0.8645537008427937|
|  2| 0.9031701155118229|  1.2524569684217643|
|  3|0.09430205113458567|  -2.573636861034734|
|  4|0.38340505276222947|  0.5469737451926588|
|  5| 0.1982919638208397| 0.06157382353970104|
|  6|0.12714181165849525|  0.3623040918178586|
|  7| 0.7604318153406678|-0.49575204523675975|
|  8|   0.83487085888236|   1.022815424084479|
|  9| 0.3142596916968412|   2.750429557170309|
+---+-------------------+--------------------+



#### 2. Summary and Descriptive Statistics


In [17]:
df.describe().show()

+-------+------------------+-------------------+------------------+
|summary|                id|            uniform|            normal|
+-------+------------------+-------------------+------------------+
|  count|                10|                 10|                10|
|   mean|               4.5| 0.4760757936207261|0.4380572306095948|
| stddev|3.0276503540974917| 0.3055791028722139|1.3604956570524473|
|    min|                 0|0.09430205113458567|-2.573636861034734|
|    max|                 9| 0.9031701155118229| 2.750429557170309|
+-------+------------------+-------------------+------------------+



In [18]:
df.describe('uniform', 'normal').show()

+-------+-------------------+------------------+
|summary|            uniform|            normal|
+-------+-------------------+------------------+
|  count|                 10|                10|
|   mean| 0.4760757936207261|0.4380572306095948|
| stddev| 0.3055791028722139|1.3604956570524473|
|    min|0.09430205113458567|-2.573636861034734|
|    max| 0.9031701155118229| 2.750429557170309|
+-------+-------------------+------------------+



Minimo, maximo, media por fila si lo ponemos o de todo el df si no ponemos fila:

In [19]:
 df.select(F.mean('uniform'), F.min('uniform'), F.max('uniform')).show() 

+------------------+-------------------+------------------+
|      avg(uniform)|       min(uniform)|      max(uniform)|
+------------------+-------------------+------------------+
|0.4760757936207261|0.09430205113458567|0.9031701155118229|
+------------------+-------------------+------------------+



En muchas ocasiones, es adecuado renombrar la nueva columna:

In [20]:
#ALIAS PARA RENOMBRAR 
df.select(

     F.mean('uniform').alias("media"),
     F.min('uniform').alias("minimo"),
     F.max('uniform').alias("maximo")

 ).show()

+------------------+-------------------+------------------+
|             media|             minimo|            maximo|
+------------------+-------------------+------------------+
|0.4760757936207261|0.09430205113458567|0.9031701155118229|
+------------------+-------------------+------------------+



#### 3. Sample covariance and correlation

*withColumn crea una columna y si existe la sobre-escribe

In [21]:
df = (

    spark.range(0, 10)
    .withColumn('rand1', F.rand(seed=10))
    .withColumn('rand2', F.rand(seed=27))

)

In [22]:
df.stat.cov('rand1', 'rand2')

0.05983805032757693

In [23]:
df.stat.cov('id', 'id')

9.166666666666666

In [24]:
df.stat.corr('rand1', 'rand2')

0.6397807763656534

In [25]:
df.stat.corr('id', 'id')

1.0

#### 4. Cross Tabulation (Contingency Table)

In [26]:
# Create a DataFrame with two columns (name, item)
names = ["Alice", "Bob", "Mike"]
items = ["milk", "bread", "butter", "apples", "oranges"]

In [27]:
pares = [(names[i % 3], items[i % 5]) for i in range(100)]

In [28]:
pares[:5]

[('Alice', 'milk'),
 ('Bob', 'bread'),
 ('Mike', 'butter'),
 ('Alice', 'apples'),
 ('Bob', 'oranges')]

In [29]:
df = spark.createDataFrame(pares,["name", "item"])

In [30]:
df.show(5)

+-----+-------+
| name|   item|
+-----+-------+
|Alice|   milk|
|  Bob|  bread|
| Mike| butter|
|Alice| apples|
|  Bob|oranges|
+-----+-------+
only showing top 5 rows



crosstab = TABLA DE CONTEO DE 2 dimensiones

In [31]:
df.stat.crosstab("name", "item").show()

+---------+------+-----+------+----+-------+
|name_item|apples|bread|butter|milk|oranges|
+---------+------+-----+------+----+-------+
|      Bob|     6|    7|     7|   6|      7|
|     Mike|     7|    6|     7|   7|      6|
|    Alice|     7|    7|     6|   7|      7|
+---------+------+-----+------+----+-------+



#### 5. Frequent Items (parecido a crosstab pero por parejas)

In [32]:
df = spark.createDataFrame(
    [(1, 2, 3) if i % 2 == 0 else (i, 2 * i, i % 4) for i in range(100)],
    ["a", "b", "c"]
)

In [33]:
df.show(5)

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
+---+---+---+
only showing top 5 rows



In [34]:
freq = df.stat.freqItems(["a", "b", "c"], 0.4)

In [35]:
freq.show()

+-----------+-----------+-----------+
|a_freqItems|b_freqItems|c_freqItems|
+-----------+-----------+-----------+
|    [1, 99]|   [2, 198]|     [1, 3]|
+-----------+-----------+-----------+



#### 6. Mathematical Functions

In [36]:
df = spark.range(0, 10).withColumn('uniform', F.rand(seed=10) * 3.14) #creamos una dummy df

In [37]:
df.show()

+---+-------------------+
| id|            uniform|
+---+-------------------+
|  0| 1.2990577122386398|
|  1| 2.2958798545155346|
|  2|  2.835954162707124|
|  3|0.29610844056259905|
|  4| 1.2038918656734006|
|  5| 0.6226367663974367|
|  6| 0.3992252886076751|
|  7| 2.3877559001696973|
|  8| 2.6214944968906106|
|  9| 0.9867754319280814|
+---+-------------------+



In [38]:
(
    df
    .select(
        'uniform',
        # Convertir en grados
        F.toDegrees('uniform'),
        (

            # cos^2(x)
            F.pow(F.cos(df['uniform']), 2) + 

            # sin^2(x)
            F.pow(F.sin(df.uniform), 2)

        ).alias("cos^2 + sin^2")
    )

).show()

+-------------------+------------------+------------------+
|            uniform|  DEGREES(uniform)|     cos^2 + sin^2|
+-------------------+------------------+------------------+
| 1.2990577122386398| 74.43052425519424|               1.0|
| 2.2958798545155346| 131.5442259328496|               1.0|
|  2.835954162707124|162.48820441567537|0.9999999999999999|
|0.29610844056259905| 16.96576392243732|0.9999999999999999|
| 1.2038918656734006| 68.97792289321647|               1.0|
| 0.6226367663974367| 35.67445888424608|0.9999999999999999|
| 0.3992252886076751|22.873924112112007|               1.0|
| 2.3877559001696973| 136.8083355871844|               1.0|
| 2.6214944968906106| 150.2005706886031|               1.0|
| 0.9867754319280814|56.538067576677925|               1.0|
+-------------------+------------------+------------------+



## Combinar DataFrames

![](img/join-types.png)

Podemos distinguir dos manearas de combinar dos `DF`:

* **`union`**: Muy simple, los dos `DF`tienen que tener el mismo número de columnas y se combinan simplemente poniendo uno encima de otro. Similar al `UNION ALL` de *SQL* o al `rbind` de *R*.


* **`join`**: Nos permite cruzar información de dos `DF` por una o más condiciones sobre sus columnas.


Veamos algunos ejemplos de la versatilidad de `join`:


In [39]:
empleados = spark.createDataFrame([

    ("Rafferty", 31),
    ("Jones", 33),
    ("Heisenberg", 33),
    ("Robinson", 34),
    ("Smith", 34),
    ("Williams", None)

],schema=["LastName", "DepartmentID"])

In [40]:
departamentos = spark.createDataFrame([

    (31, "Sales"), 
    (33, "Engineering"), 
    (34, "Clerical"),
    (35, "Marketing")

],schema=["DepartmentID", "DepartmentName"])

In [41]:
empleados.show()

+----------+------------+
|  LastName|DepartmentID|
+----------+------------+
|  Rafferty|          31|
|     Jones|          33|
|Heisenberg|          33|
|  Robinson|          34|
|     Smith|          34|
|  Williams|        null|
+----------+------------+



In [42]:
departamentos.show()

+------------+--------------+
|DepartmentID|DepartmentName|
+------------+--------------+
|          31|         Sales|
|          33|   Engineering|
|          34|      Clerical|
|          35|     Marketing|
+------------+--------------+



In [43]:
(

    empleados
    .join(departamentos,'DepartmentID') #POR DEFECTO HACE INNER JOIN

).show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
|          31|  Rafferty|         Sales|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
+------------+----------+--------------+



In [44]:
(

    empleados
    .join(departamentos,'DepartmentID','left') #LEFT

).show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
|          31|  Rafferty|         Sales|
|        null|  Williams|          null|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
+------------+----------+--------------+



In [45]:
(

    empleados
    .join(departamentos,'DepartmentID','full') #FULL

).show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
|          31|  Rafferty|         Sales|
|        null|  Williams|          null|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
|          35|      null|     Marketing|
+------------+----------+--------------+



In [46]:
(

    empleados
    .crossJoin(departamentos)
    

).show()
#intentar evitar que dos columnas se llamen igua!!! 
    #si no al seleccionar nos da pronlemas
    #.crossJoin(departamentos.withColumnRename()) - mirar online

+----------+------------+------------+--------------+
|  LastName|DepartmentID|DepartmentID|DepartmentName|
+----------+------------+------------+--------------+
|  Rafferty|          31|          31|         Sales|
|  Rafferty|          31|          33|   Engineering|
|     Jones|          33|          31|         Sales|
|     Jones|          33|          33|   Engineering|
|Heisenberg|          33|          31|         Sales|
|Heisenberg|          33|          33|   Engineering|
|  Rafferty|          31|          34|      Clerical|
|  Rafferty|          31|          35|     Marketing|
|     Jones|          33|          34|      Clerical|
|     Jones|          33|          35|     Marketing|
|Heisenberg|          33|          34|      Clerical|
|Heisenberg|          33|          35|     Marketing|
|  Robinson|          34|          31|         Sales|
|  Robinson|          34|          33|   Engineering|
|     Smith|          34|          31|         Sales|
|     Smith|          34|   

¿Qué pasa si hay duplicados en la clave de cruce?

In [47]:
departamentos2 = spark.createDataFrame([

    (31, "Sales"), 
    (33, "Engineering"), 
    (34, "Clerical"),
    (35, "Marketing"),
    (31, "Ventas"),

],schema=["DepartmentID", "DepartmentName"])

In [48]:
(

    empleados
    .join(departamentos2,'DepartmentID')

).show()

+------------+----------+--------------+
|DepartmentID|  LastName|DepartmentName|
+------------+----------+--------------+
|          34|  Robinson|      Clerical|
|          34|     Smith|      Clerical|
|          31|  Rafferty|         Sales|
|          31|  Rafferty|        Ventas|
|          33|     Jones|   Engineering|
|          33|Heisenberg|   Engineering|
+------------+----------+--------------+



Por último, veamos como la condición del `join` puede ser mucho más compleja:

In [49]:
productos = spark.createDataFrame([

  ("steak", "1990-01-01", "2000-01-01", 150),
  ("steak", "2000-01-02", "2020-01-01", 180),
  ("fish", "1990-01-01", "2020-01-01", 100)

],schema=["name", "startDate", "endDate", "price"])

In [50]:
pedidos = spark.createDataFrame([

  ("1995-01-01", "steak"),
  ("2000-01-01", "fish"),
  ("2005-01-01", "steak")

],schema=["date", "product"])

In [51]:
#JOIN MUY COMPLEJO QUE SOLO SE HACE CUANDO LA SENTENCIA ES TRUE
(

    pedidos
    .join(
        productos,
        ( F.col('product') == F.col('name') ) &
        ( F.col('date') >= F.col('startDate') ) &
        ( F.col('date') <= F.col('endDate') )    
    )
  
).show()

+----------+-------+-----+----------+----------+-----+
|      date|product| name| startDate|   endDate|price|
+----------+-------+-----+----------+----------+-----+
|1995-01-01|  steak|steak|1990-01-01|2000-01-01|  150|
|2005-01-01|  steak|steak|2000-01-02|2020-01-01|  180|
|2000-01-01|   fish| fish|1990-01-01|2020-01-01|  100|
+----------+-------+-----+----------+----------+-----+



## Agregaciones

Ya hemos visto el commando `groupBy` que nos sirve para agregar por ciertas columnas. Veremos algunos ejemplos de qué podemos hacer agrupando por columnas:

*Como definir un esquema a mano:

In [52]:
esquema = T.StructType([

    T.StructField("carat",T.DoubleType()),
    T.StructField("cut",T.StringType()),
    T.StructField("color",T.StringType()),
    T.StructField("clarity",T.StringType()),
    T.StructField("depth",T.DoubleType()),
    T.StructField("table",T.DoubleType()),
    T.StructField("price",T.IntegerType()),
    T.StructField("x",T.DoubleType()),
    T.StructField("y",T.DoubleType()),
    T.StructField("z",T.DoubleType()),

])

Volvemos a cargar el dataset `diamonds.csv` en este caso ya damos el esquema definido:

In [53]:
diamonds = (
    
    spark.read
    .options(header=True)
    .schema(esquema) ##AQUI LO PONEMOS EL ESQUEMA
    .csv('/datos/diamonds.csv')

).cache()

In [54]:
diamonds.printSchema()

root
 |-- carat: double (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: double (nullable = true)
 |-- table: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [55]:
diamonds.groupBy("cut").count().show() #group by y luego count/sum/agg(cualquier funcion de tipo agregacion)

+---------+-----+
|      cut|count|
+---------+-----+
|  Premium|13791|
|    Ideal|21551|
|     Good| 4906|
|     Fair| 1610|
|Very Good|12082|
+---------+-----+



**Por defecto el group by no ordena por nada

In [56]:
diamonds.groupBy("cut").agg(F.count("*").alias("conteo")).show()

+---------+------+
|      cut|conteo|
+---------+------+
|  Premium| 13791|
|    Ideal| 21551|
|     Good|  4906|
|     Fair|  1610|
|Very Good| 12082|
+---------+------+



In [57]:
diamonds.show(4)

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
| 0.21|Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
| 0.23|   Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
| 0.29|Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
only showing top 4 rows



Con `agg` Podemos crear varias columnas en la misma agrupación:

In [13]:
agregado = (

    diamonds
    .groupBy("cut")
    .agg(
        F.count("*").alias("conteo"),
        F.sum('x').alias("x"),
        F.sum('y').alias("y"),
        F.sum('z').alias("z"),
        F.mean('price').alias('media'),
        F.collect_set('clarity').alias('clarity') #COLLECT SET METE TODOS LOS VALORES DISTINTOS EN UNA LISTA
    )

)

NameError: name 'diamonds' is not defined

In [14]:
agregadp #AUNQUE NO HAY HECHO NADA SABE QUE TIPO VA A RESULTAR

NameError: name 'agregadp' is not defined

In [59]:
agregado.toPandas()

,cut,conteo,x,y,z,media,clarity
0,Premium,13791,82385.88,82385.88,82385.88,4584.257704,"[VS2, SI2, IF, VS1, VVS2, VVS1, I1, SI1]"
1,Ideal,21551,118691.07,118691.07,118691.07,3457.541970,"[VS2, SI2, IF, VS1, VVS2, VVS1, I1, SI1]"
2,Good,4906,28645.08,28645.08,28645.08,3928.864452,"[VS2, SI2, IF, VS1, VVS2, VVS1, I1, SI1]"
3,Fair,1610,10057.50,10057.50,10057.50,4358.757764,"[SI1, SI2, IF, VS1, VVS2, VVS1, I1, VS2]"
4,Very Good,12082,69359.09,69359.09,69359.09,3981.759891,"[VS2, SI2, IF, VS1, VVS2, VVS1, I1, SI1]"


In [60]:
diamonds.unpersist()

DataFrame[carat: double, cut: string, color: string, clarity: string, depth: double, table: double, price: int, x: double, y: double, z: double]

## Funciones Ventanas-Windows

![](img/windows.png)

&nbsp;

Las funciones de tipo [*ventana*](https://en.wikipedia.org/wiki/SQL_window_function) nos permite hacer operaciones analíticas más complejas y usar información de otros registros. Veamos algunos ejemplos:

In [21]:
productRevenue = spark.read.load('/datos/productRevenue.parquet').cache() #PARQUE YA LLEVA DENTRO EL ESQUEMA
        #EL ESQUEMA VIENE DENTRO DEL ARCHIVO
        #ADEMAS ES DE TIPO COLUMNAL

In [16]:
productRevenue.count()

10

In [63]:
productRevenue.show()

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Thin|Cell phone|   6000|
|    Normal|    Tablet|   1500|
|      Mini|    Tablet|   5500|
|Ultra Thin|Cell phone|   5000|
| Very Thin|Cell phone|   6000|
|       Big|    Tablet|   2500|
|  Bendable|Cell phone|   3000|
|  Foldable|Cell phone|   3000|
|       Pro|    Tablet|   4500|
|      Pro2|    Tablet|   6500|
+----------+----------+-------+



Queremos calcular:

* La difrencia entre el mayor ingreso de la categoría y el actual.
* El puesto (1º,2º,...) a nivel de ingresos para cada categoría.

In [19]:
from pyspark.sql.window import Window

*Window crea una columna cn el mismo numero de entradas pero haciendo funciones por agrupados. Como un groupby pero mostrando todo 

In [20]:
(

    productRevenue
    .withColumn(
        "revenue_max",
        F.max('revenue').over(Window.partitionBy('category')) #DAR EL MAXIMO REV PARTICIONADO POR CATEGORIA 
    )
    .withColumn("revenue_difference", F.col('revenue_max') - F.col('revenue'))

).show()

+----------+----------+-------+-----------+------------------+
|   product|  category|revenue|revenue_max|revenue_difference|
+----------+----------+-------+-----------+------------------+
|      Thin|Cell phone|   6000|       6000|                 0|
|Ultra Thin|Cell phone|   5000|       6000|              1000|
| Very Thin|Cell phone|   6000|       6000|                 0|
|  Bendable|Cell phone|   3000|       6000|              3000|
|  Foldable|Cell phone|   3000|       6000|              3000|
|    Normal|    Tablet|   1500|       6500|              5000|
|      Mini|    Tablet|   5500|       6500|              1000|
|       Big|    Tablet|   2500|       6500|              4000|
|       Pro|    Tablet|   4500|       6500|              2000|
|      Pro2|    Tablet|   6500|       6500|                 0|
+----------+----------+-------+-----------+------------------+



Además de las funciones que ya hemos visto, también podemos usar las siguientes funciones sobre una ventana:

<table class="table">
<tbody>
<tr>
<td></td>
<td><strong>SQL</strong></td>
<td><strong>DataFrame API</strong></td>
</tr>
<tr>
<td rowspan="5"><strong>Ranking functions</strong></td>
<td>rank</td>
<td>rank</td>
</tr>
<tr>
<td>dense_rank</td>
<td>denseRank</td>
</tr>
<tr>
<td>percent_rank</td>
<td>percentRank</td>
</tr>
<tr>
<td>ntile</td>
<td>ntile</td>
</tr>
<tr>
<td>row_number</td>
<td>rowNumber</td>
</tr>
<tr>
<td rowspan="5"><strong>Analytic functions</strong></td>
<td>cume_dist</td>
<td>cumeDist</td>
</tr>
<tr>
<td>first_value</td>
<td>firstValue</td>
</tr>
<tr>
<td>last_value</td>
<td>lastValue</td>
</tr>
<tr>
<td>lag</td>
<td>lag</td>
</tr>
<tr>
<td>lead</td>
<td>lead</td>
</tr>
</tbody>
</table>

Calculemos ahora el ranqueo pedido:

#### Diferencia entre rank y frank:
- Rank te empata si hay empates (1,1,3...)
- DenseRank si hay empate no salta (1,1,2...)
- row_number (1,2,3...)

In [22]:
(

    productRevenue
    .withColumn(
        "ranking",
        F.rank() #no tiene argumentos
        .over(
            Window
            .partitionBy('category') #ventana particionada por categoria
            .orderBy(F.desc('revenue')) #ordenada por revenue
        )
    )
    .withColumn(
        "dense_ranking",
        F.dense_rank()
        .over(
            Window
            .partitionBy('category')
            .orderBy(F.desc('revenue'))
        )
    )
    .withColumn(
        "row_number",
        F.row_number()
        .over(
            Window
            .partitionBy('category')
            .orderBy(F.desc('revenue'))
        )
    )

).show()

+----------+----------+-------+-------+-------------+----------+
|   product|  category|revenue|ranking|dense_ranking|row_number|
+----------+----------+-------+-------+-------------+----------+
|      Thin|Cell phone|   6000|      1|            1|         1|
| Very Thin|Cell phone|   6000|      1|            1|         2|
|Ultra Thin|Cell phone|   5000|      3|            2|         3|
|  Bendable|Cell phone|   3000|      4|            3|         4|
|  Foldable|Cell phone|   3000|      4|            3|         5|
|      Pro2|    Tablet|   6500|      1|            1|         1|
|      Mini|    Tablet|   5500|      2|            2|         2|
|       Pro|    Tablet|   4500|      3|            3|         3|
|       Big|    Tablet|   2500|      4|            4|         4|
|    Normal|    Tablet|   1500|      5|            5|         5|
+----------+----------+-------+-------+-------------+----------+



Podemos definir la ventana por separado para simplificar el código:

In [26]:
ventana = (

    Window
    .partitionBy('category')
    .orderBy(F.desc('revenue'))

)

In [27]:
ranqueados = (

    productRevenue
    .withColumn("ranking",F.rank().over(ventana))
    .withColumn("dense_ranking",F.dense_rank().over(ventana))

)

In [69]:
ranqueados.show()

+----------+----------+-------+-------+-------------+
|   product|  category|revenue|ranking|dense_ranking|
+----------+----------+-------+-------+-------------+
|      Thin|Cell phone|   6000|      1|            1|
| Very Thin|Cell phone|   6000|      1|            1|
|Ultra Thin|Cell phone|   5000|      3|            2|
|  Bendable|Cell phone|   3000|      4|            3|
|  Foldable|Cell phone|   3000|      4|            3|
|      Pro2|    Tablet|   6500|      1|            1|
|      Mini|    Tablet|   5500|      2|            2|
|       Pro|    Tablet|   4500|      3|            3|
|       Big|    Tablet|   2500|      4|            4|
|    Normal|    Tablet|   1500|      5|            5|
+----------+----------+-------+-------+-------------+



## Data Sources

![](img/blog-illustration-01.png)

Desde spark podemos leer todo tipo de formatos, veamos algunos ejemplos más avanzados. Lo primero de todo vamos a cambiar cada uno a la databse de hive de nuestro usuario:

In [23]:
mi_user = os.environ.get('USER') #os nos da nuestro usuario 
print(mi_user)

jhelena


In [24]:
spark.catalog.setCurrentDatabase(mi_user) #para cambiar la base de datos por defecto a la nuestra

Guardamos los datos en Hive:
**Por defecto guarda en formato PARQUET (LO MAS COMODO EN SPARK)

In [28]:
ranqueados.write.saveAsTable("ranqueos") #NOS DARIA ERROR SI YA LA TUVIERAMOS

In [29]:
pd.DataFrame(spark.catalog.listTables())

,name,database,description,tableType,isTemporary
0,airport_part,jhelena,None,EXTERNAL,False
1,books,jhelena,None,EXTERNAL,False
2,books_part,jhelena,None,EXTERNAL,False
3,dns,jhelena,None,EXTERNAL,False
4,ranqueos,jhelena,None,MANAGED,False
5,ratings_parquet,jhelena,None,MANAGED,False
6,ratings_parquet_snappy,jhelena,None,MANAGED,False
7,restaurants2019,jhelena,None,MANAGED,False
8,terrorism,jhelena,None,EXTERNAL,False
9,terrorism_age_groups,jhelena,None,MANAGED,False


También podemos borrar la tabla creada con sentencia *SQL*:

In [30]:
spark.sql(" DROP TABLE ranqueos")

DataFrame[]

In [31]:
pd.DataFrame(spark.catalog.listTables())

,name,database,description,tableType,isTemporary
0,airport_part,jhelena,None,EXTERNAL,False
1,books,jhelena,None,EXTERNAL,False
2,books_part,jhelena,None,EXTERNAL,False
3,dns,jhelena,None,EXTERNAL,False
4,ratings_parquet,jhelena,None,MANAGED,False
5,ratings_parquet_snappy,jhelena,None,MANAGED,False
6,restaurants2019,jhelena,None,MANAGED,False
7,terrorism,jhelena,None,EXTERNAL,False
8,terrorism_age_groups,jhelena,None,MANAGED,False
9,terroristplot,jhelena,None,EXTERNAL,False


Además podemos guardar la tabla directamen en un direcotrio del hdfs:

In [76]:
ranqueados.write.parquet("ranqueos.parquet") #guarda en el hdfs en parquet

In [32]:
ranqueados.write.save("ranqueos.parquet") #save guarda por lo que hay por defecto (parquet)

En este caso tanto `parquet` como `save` hace lo mismo guardar los datos en formato parquet. La última sentencia nos ha dado error porque ya exitía el archivo.

### Save Modes
Por defecto al guardar un `DF` obtenemos error si existe el directorio, podemos configurar para que esto no ocurra:

<table class="table">
<tbody><tr><th>Scala/Java</th><th>Any Language</th><th>Meaning</th></tr>
<tr>
  <td><code>SaveMode.ErrorIfExists</code> (default)</td>
  <td><code>"error"</code> (default)</td>
  <td>
    When saving a DataFrame to a data source, if data already exists,
    an exception is expected to be thrown.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Append</code></td>
  <td><code>"append"</code></td>
  <td>
    When saving a DataFrame to a data source, if data/table already exists,
    contents of the DataFrame are expected to be appended to existing data.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Overwrite</code></td>
  <td><code>"overwrite"</code></td>
  <td>
    Overwrite mode means that when saving a DataFrame to a data source,
    if data/table already exists, existing data is expected to be overwritten by the contents of
    the DataFrame.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Ignore</code></td>
  <td><code>"ignore"</code></td>
  <td>
    Ignore mode means that when saving a DataFrame to a data source, if data already exists,
    the save operation is expected to not save the contents of the DataFrame and to not
    change the existing data. This is similar to a <code>CREATE TABLE IF NOT EXISTS</code> in SQL.
  </td>
</tr>
</tbody></table>

In [33]:
ranqueados.write.mode('overwrite').save("ranqueos.parquet") #TIRA LO QUE HAYA Y LO VUELVE A GUARDAR

In [79]:
spark.read.load("ranqueos.parquet").count() 

10

Con `append` añadimos a la tabla existente:

In [34]:
for i in range(3):
    ranqueados.write.mode('append').save("ranqueos.parquet") 
    
#ANADIMOS A LA TABLA TRES VECES

In [35]:
spark.read.load("ranqueos.parquet").count()

40

### JSON

![](img/json.png)

También podemos leer y escribir en formato JSON, aunque realmente se usa el formato [*JSON lines*](http://jsonlines.org/) dónde cada línea es un registro:

In [82]:
!hadoop fs -text /datos/restaurants.json | head -n 3
#el json de big data va cada registro es una fila = json lines

{"URL":"http://www.just-eat.co.uk/restaurants-albany-spice/menu","_id":{"$oid":"55f14312c7447c3da7051d42"},"address":"Stella Building","address_line":"Washington","name":"Albany Spice","outcode":"NE37","postcode":"1BH","rating":4.5,"type_of_food":"Curry"}
{"URL":"http://www.just-eat.co.uk/restaurants-albarakah-hd4/menu","_id":{"$oid":"55f14312c7447c3da7051d43"},"address":"279 Manchester Road","address_line":"West Yorkshire","name":"Albarakah","outcode":"HD4","postcode":"5AA","rating":4.5,"type_of_food":"Curry"}
{"URL":"http://www.just-eat.co.uk/restaurants-albatta-co1/menu","_id":{"$oid":"55f14312c7447c3da7051d44"},"address":"18 Sir Isaac Walk","address_line":"Colchester","name":"Albatta","outcode":"CO1","postcode":"1JJ","rating":5.0,"type_of_food":"Lebanese"}
text: Unable to write to output stream.


In [36]:
restaurantes = spark.read.json('/datos/restaurants.json') #EN JSON YA VIENEN LOS ESQUEMAS

*Pasar todo a parquet que va MUCHO MAS RAPIDO

In [37]:
restaurantes.printSchema()

root
 |-- URL: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- address: string (nullable = true)
 |-- address_line: string (nullable = true)
 |-- name: string (nullable = true)
 |-- outcode: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- type_of_food: string (nullable = true)



In [85]:
restaurantes.limit(10).toPandas()

,URL,_id,address,address_line,name,outcode,postcode,rating,type_of_food
0,http://www.just-eat.co.uk/restaurants-albany-s...,"(55f14312c7447c3da7051d42,)",Stella Building,Washington,Albany Spice,NE37,1BH,4.5,Curry
1,http://www.just-eat.co.uk/restaurants-albaraka...,"(55f14312c7447c3da7051d43,)",279 Manchester Road,West Yorkshire,Albarakah,HD4,5AA,4.5,Curry
2,http://www.just-eat.co.uk/restaurants-albatta-...,"(55f14312c7447c3da7051d44,)",18 Sir Isaac Walk,Colchester,Albatta,CO1,1JJ,5.0,Lebanese
3,http://www.just-eat.co.uk/restaurants-albertos...,"(55f14312c7447c3da7051d45,)",112 Gannow Lane,Burnley,Alberto's Pizza & Kebab House,BB12,6QD,5.5,Kebab
4,http://www.just-eat.co.uk/restaurants-albertsf...,"(55f14312c7447c3da7051d46,)",746 City Road,Sheffield,Albert's Fish & Chips,S2,1GN,4.5,Fish & Chips
5,http://www.just-eat.co.uk/restaurants-albions-...,"(55f14312c7447c3da7051d47,)",120 Midland Road Royston,Barnsley,Albions,S71,4QT,5.0,Pizza
6,http://www.just-eat.co.uk/restaurants-albions-...,"(55f14312c7447c3da7051d48,)",120 Midland Road Royston,Barnsley,Albions,S71,4QT,5.0,Pizza
7,http://www.just-eat.co.uk/restaurants-alborz/menu,"(55f14312c7447c3da7051d49,)",71 Ormskirk Road,Wigan,Alborz,WN5,9EA,5.5,American
8,http://www.just-eat.co.uk/restaurants-albright...,"(55f14312c7447c3da7051d4a,)",5/6 High Street,Albrighton,Albrighton Balti,WV7,3JX,5.5,Curry
9,http://www.just-eat.co.uk/restaurants-alcapone...,"(55f14312c7447c3da7051d4b,)",105 Highfield Road,Blackpool,Alcapone Pizza Shop,FY4,2JE,5.5,Pizza


### JDBC (Java Database Connectivity)

![](img/java_jdbc.png)

JDBC es un protocolo altamente utilizado por las bases de datos. De esta manera, muchas base de dato tiene un *driver* para este protocolo. Veamos por ejemplo como conectarnos a *MySQL*

**NOTA:** Observar como al iniciar la sesión de spark pusimos:



```python
.set("spark.jars","/var/lib/sqoop/mysql-connector-java-5.1.44-bin.jar")
```
Para añadir el driver a la sesión.


In [86]:
tags = (

    spark.read
    .format("jdbc") #FORMATO TIPO JDBC
    .option("url", "jdbc:mysql://192.168.80.33:3306/movielens") #DRIVER QUE VAMOS A USAR, HOST, Y BASE DE DATOS
    .option("driver", "com.mysql.jdbc.Driver") 
    .option("dbtable", "tags") #LA TABLA QUE QUIERES LEER
    .option("user", "root")
    .option("password", "root")
    .load()
 
)
#no lee los datos pero ve lo que hay y crea el esquema

In [87]:
tags.count()

174846

In [88]:
tags.show()

+------+-------+-----------------+----------+
|userId|movieId|              tag|        ts|
+------+-------+-----------------+----------+
|    18|   4141|      Mark Waters|1240597180|
|    65|    208|        dark hero|1368150078|
|    65|    353|        dark hero|1368150079|
|    65|    521|    noir thriller|1368149983|
|    65|    592|        dark hero|1368150078|
|    65|    668|        bollywood|1368149876|
|    65|    898| screwball comedy|1368150160|
|    65|   1248|    noir thriller|1368149983|
|    65|   1391|             mars|1368150055|
|    65|   1617|         neo-noir|1368150217|
|    65|   1694|            jesus|1368149925|
|    65|   1783|    noir thriller|1368149983|
|    65|   2022|            jesus|1368149925|
|    65|   2193|           dragon|1368151314|
|    65|   2353|conspiracy theory|1368151266|
|    65|   2662|             mars|1368150055|
|    65|   2726|    noir thriller|1368149983|
|    65|   2840|            jesus|1368149925|
|    65|   3052|            jesus|

In [89]:
(

    tags
    .groupBy('movieID','tag')
    .count()
    .filter('count>2')
    .orderBy(F.desc('count'))

).show()

+-------+------------------+-----+
|movieID|               tag|count|
+-------+------------------+-----+
|  79132| alternate reality|   98|
|   2959|         Brad Pitt|   74|
|    593|   Anthony Hopkins|   64|
|   1653|          dystopia|   54|
|   2420|            mentor|   53|
|  70286|            aliens|   52|
|   4973|beautifully filmed|   52|
|  79132|            action|   50|
|   4226|         nonlinear|   49|
|   8874|      black comedy|   49|
|   2291|          original|   48|
|  44191|          dystopia|   48|
|     32|         Brad Pitt|   48|
|   2329|     Edward Norton|   47|
|   6711|       atmospheric|   47|
|  64614|    Clint Eastwood|   46|
|  69122|            comedy|   46|
|   5618|             anime|   46|
|     47|         Brad Pitt|   45|
|  58559|            Batman|   45|
+-------+------------------+-----+
only showing top 20 rows



In [90]:
resumen_pelis = (

    tags
    .groupBy('movieID')
    .agg(F.collect_set('tag').alias('tags'))
    .withColumn('conteo',F.size('tags')) #F.size te dice el numero de elementos en una lista

)

In [91]:
resumen_pelis.show()

+-------+--------------------+------+
|movieID|                tags|conteo|
+-------+--------------------+------+
|    148|[nudity (topless)...|     2|
|    471|[VHS, Business is...|    27|
|    496|[Sundance award w...|     2|
|    833|[ZAZ, comedy, spo...|     4|
|   1088|[coming of age, g...|    20|
|   1238|[affectionate, Be...|     8|
|   1342|[gruesome, CLV, n...|    13|
|   1580|[comic book, Acti...|    35|
|   1591|[super hero, comi...|     7|
|   1645|[courtroom, DVD, ...|    26|
|   1829|[deep, China, Hon...|     3|
|   1959|[Oscar (Best Cine...|    17|
|   2122|[Eric's Dvds, ada...|    14|
|   2142|[adventure, class...|     3|
|   2366|[stop motion, dvd...|    26|
|   2866|[Steve Rash, musi...|     4|
|   3175|[star trek, DVD-V...|    25|
|   3749|[John Malkovich, ...|     2|
|   3794|             [indie]|     1|
|   3918|[Betamax, Tumey's...|     8|
+-------+--------------------+------+
only showing top 20 rows



Podemos escribir la información procesada del *MySQL* directamente a un fichero *JSON* por ejemplo:

In [92]:
resumen_pelis.write.mode('overwrite').json('pelis.json') #guardar a json

In [93]:
for i in spark.sparkContext.textFile('pelis.json').take(4):
    print(i)

{"movieID":148,"tags":["nudity (topless)","Nudity (Topless - Notable)"],"conteo":2}
{"movieID":471,"tags":["VHS","Business is the antagonist","satirical","Comedy","funny","no idea","BD-R","f","Didn't finish","coen bros","cheesy","quirky","New Year's Eve","hula hoop","btaege","Capra-esque","NO_FA_GANES","Fantasy","Tumey's DVDs","coen brothers","Underrated","BOARDROOM JUNGLE","1950s","business","To See","netflixq","Coen Brothers"],"conteo":27}
{"movieID":496,"tags":["Sundance award winner","loneliness"],"conteo":2}
{"movieID":833,"tags":["ZAZ","comedy","spoof","Hart Bochner"],"conteo":4}


In [94]:
spark.read.json('pelis.json').show()

+------+-------+--------------------+
|conteo|movieID|                tags|
+------+-------+--------------------+
|     8|     35|[bisexual, compar...|
|     2|    503|[social satire, l...|
|     1|    583|     [Nanni Moretti]|
|    29|    594|[adapted from:boo...|
|     9|    610|[Adult Animation,...|
|     6|    761|[pretty terrible,...|
|     8|    880|[Futuristmovies.c...|
|     1|   1519|[less than 300 ra...|
|    18|   1589|[pretty good, dra...|
|     1|   1815|[less than 300 ra...|
|     9|   1881|[Nostalgia Critic...|
|    25|   2080|[Disney, reviewed...|
|    46|   2324|[father, top 250 ...|
|     1|   2444|   [black and white]|
|     3|   2445|[blindness, Irwin...|
|     5|   2649|[atmosphere, Rowl...|
|    13|   2915|[male teenager(s)...|
|     1|   3065|[less than 300 ra...|
|     7|   3316|[nudity (topless)...|
|    21|   3468|[Oscar (Best Cine...|
+------+-------+--------------------+
only showing top 20 rows



In [38]:
spark.stop()